In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import cv2 as cv2
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm
import os
import re
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
import tensorflow as tf

In [ ]:
#Counting Files

image_folder = '/content/drive/MyDrive/dataset-iu/dataset-iu-img'
#APP_FOLDER = ""
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(image_folder):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1

print('Total number of files',totalFiles)

Searching in :  /content/drive/MyDrive/dataset-iu/dataset-iu-img
Total number of files 7472


In [ ]:
text_folder = '/content/drive/MyDrive/dataset-iu/ecgen-radiology'
#APP_FOLDER = ""
totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(text_folder):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1

print('Total number of files',totalFiles)

Searching in :  /content/drive/MyDrive/dataset-iu/ecgen-radiology
Total number of files 3955


In [ ]:
columns = ["image_id", "caption", "comparison", "indication", "findings", "impression","mesh","height","width"]
dataset = pd.DataFrame(columns = columns)

In [ ]:
#extracting data from xml report

for file in tqdm(os.listdir(text_folder)):
    if file.endswith(".xml"):
        k = text_folder + '/'
        path = k + file
        mytree = ET.parse(path)# parsing xml report
        comparision = mytree.find(".//AbstractText[@Label='COMPARISON']").text # extracting comaparison text
        indication = mytree.find(".//AbstractText[@Label='INDICATION']").text #extracting indication text
        findings = mytree.find(".//AbstractText[@Label='FINDINGS']").text # extracting findings text
        impression = mytree.find(".//AbstractText[@Label='IMPRESSION']").text  # extracting impression text
        # mesh = mytree.find(".//MeSH//major").text  # extracting MeSH text
        mesh=''
        for meshElement in mytree.findall(".//MeSH//major"):  # extracting MeSH text
          mesh+=meshElement.text+' '
        
        mytree = ET.parse(path)
        for x in mytree.findall("parentImage"):
            image_id = x.attrib['id']+".png"
            filename = image_folder + '/' + image_id
            image = cv2.imread(filename) # reading image
            #os.path.exists(filename)
             
            height, width, channels = image.shape
            caption = '' if x.find('caption').text is None else x.find('caption').text
     
            dataset = dataset.append(pd.Series([image_id, caption, comparision, indication, findings, impression,mesh,height,width],index = columns), ignore_index = True)

  0%|          | 0/3955 [00:00<?, ?it/s]

In [ ]:
dataset.shape

(7470, 9)

In [ ]:
dataset

,image_id,caption,comparison,indication,findings,impression,mesh,height,width
0,CXR3695_IM-1845-1001.png,Two views of the chest,XXXX,Shortness of breath,The cardiac silhouette size is at the upper li...,"1. Mild central vascular prominence, XXXX cong...",Pulmonary Congestion/mild,420,512
1,CXR3695_IM-1845-2001.png,Two views of the chest,XXXX,Shortness of breath,The cardiac silhouette size is at the upper li...,"1. Mild central vascular prominence, XXXX cong...",Pulmonary Congestion/mild,624,512
2,CXR3703_IM-1850-1001.png,"Chest 2 views dated XXXX, XXXX",None.,History of atrial fibrillation and cardioversi...,The XXXX examination consists of frontal and l...,No evidence of acute cardiopulmonary process.,normal,512,512
3,CXR3703_IM-1850-2001.png,"Chest 2 views dated XXXX, XXXX",None.,History of atrial fibrillation and cardioversi...,The XXXX examination consists of frontal and l...,No evidence of acute cardiopulmonary process.,normal,547,512
4,CXR3686_IM-1837-0001-0001.png,"PA and lateral chest, XXXX, XXXX XXXX XXXX","XXXX, XXXX","Hypoxia, aortic stenosis",None,Stable cardiomegaly. Improved aeration in the ...,Cardiomegaly Airspace Disease/lung/base/right ...,423,512
...,...,...,...,...,...,...,...,...,...
7465,CXR1842_IM-0545-1002.png,"PA and lateral chest XXXX, XXXX at XXXX compar...",None,History of dyspnea,None,Heart size normal. Lungs clear. Stable 5 mm ca...,Nodule/lung/middle lobe/right Calcinosis/lung/...,513,512
7466,CXR1863_IM-0558-1001.png,Xray Chest PA and Lateral,None available.,Left-sided chest pain,Heart size is mildly enlarged. Tortuous aorta....,1. Low volume study without acute process. 2. ...,Cardiomegaly/mild Aorta/tortuous Lung/hypoinfl...,456,512
7467,CXR1863_IM-0558-3001.png,Xray Chest PA and Lateral,None available.,Left-sided chest pain,Heart size is mildly enlarged. Tortuous aorta....,1. Low volume study without acute process. 2. ...,Cardiomegaly/mild Aorta/tortuous Lung/hypoinfl...,486,512
7468,CXR186_IM-0558-1001.png,PA and lateral chest radiograph on XXXX at 00:...,None available.,"XXXX year old woman, question swallowed foreig...","Cardiac size, mediastinal contour, and pulmona...",No acute cardiopulmonary abnormalities. No evi...,normal,420,512


In [ ]:
# making the paths absolute
def absolute_path(x):
    x = image_folder + '/' + x
    return x
    
dataset['Image_path'] = dataset['image_id'].apply(lambda x : absolute_path(x))

In [ ]:
dataset[dataset["image_id"]=='CXR2_IM-0652-1001.png']

,image_id,caption,comparison,indication,findings,impression,mesh,height,width,Image_path
4113,CXR2_IM-0652-1001.png,"Chest, 2 views, frontal and lateral",None.,Preop bariatric surgery.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,Cardiomegaly/borderline Pulmonary Artery/enlar...,512,512,/content/drive/MyDrive/dataset-iu/dataset-iu-i...


In [ ]:
dataset.head(3)

,image_id,caption,comparison,indication,findings,impression,mesh,height,width,Image_path
0,CXR3695_IM-1845-1001.png,Two views of the chest,XXXX,Shortness of breath,The cardiac silhouette size is at the upper li...,"1. Mild central vascular prominence, XXXX cong...",Pulmonary Congestion/mild,420,512,/content/drive/MyDrive/dataset-iu/dataset-iu-i...
1,CXR3695_IM-1845-2001.png,Two views of the chest,XXXX,Shortness of breath,The cardiac silhouette size is at the upper li...,"1. Mild central vascular prominence, XXXX cong...",Pulmonary Congestion/mild,624,512,/content/drive/MyDrive/dataset-iu/dataset-iu-i...
2,CXR3703_IM-1850-1001.png,"Chest 2 views dated XXXX, XXXX",None.,History of atrial fibrillation and cardioversi...,The XXXX examination consists of frontal and l...,No evidence of acute cardiopulmonary process.,normal,512,512,/content/drive/MyDrive/dataset-iu/dataset-iu-i...


In [ ]:
#find all the missing values
dataset.isnull().sum()

image_id         0
caption          0
comparison    1157
indication     159
findings       997
impression      52
mesh             0
height           0
width            0
Image_path       0
dtype: int64

In [ ]:
#dataReport = df[['image_id','findings','Image_path']]
#dataClassification= df[['image_id','impression','Image_path']]

In [ ]:
#we can see that the comparison column has no use so we can drop that

In [ ]:
path = '/content/drive/My Drive/dataset-all.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  dataset.to_csv(f)

In [ ]:
img_per_person_train = combining_images(images_train)
img_per_person_cv = combining_images(images_cv)
img_per_person_test = combining_images(images_test)

NameError: ignored

In [ ]:
img_per_person_train['CXR982_IM-2470']

In [ ]:
def load_image(file):
    img = cv2.imread(file)
    return img
  
  # just checking the ID which has 4 images
for k,v in images.items():
    if v == 4:
        print(k)
        break

plt.figure(figsize=(9,9))
plt.subplot(221)
plt.imshow(load_image('/content/drive/MyDrive/dataset-iu/dataset-iu-img/CXR1102_IM-0069-12012.png'))
plt.title('CXR1102_IM-0069-12012.png')
plt.subplot(222)
plt.imshow(load_image('/content/drive/MyDrive/dataset-iu/dataset-iu-img/CXR1102_IM-0069-2001.png'))
plt.title('CXR1102_IM-0069-2001.png')
plt.subplot(223)
plt.imshow(load_image('/content/drive/MyDrive/dataset-iu/dataset-iu-img/CXR1102_IM-0069-3001.png'))
plt.title('CXR1102_IM-0069-3001.png')
plt.subplot(224)
plt.imshow(load_image('/content/drive/MyDrive/dataset-iu/dataset-iu-img/CXR1102_IM-0069-4004.png'))
plt.title('CXR1102_IM-0069-4004.png')

In [ ]:
import itertools

def create_data(image_per_person):
    # new dataset
    person_id, image1, image2, report,result = [],[],[],[],[]
    for pid, imgs in image_per_person.items():   #contains pid and the images associated with that pid

        if len(imgs) == 1:
            image1.append(imgs[0])
            image2.append(imgs[0])
            person_id.append(pid)
            report.append(findings[pid])
            result.append(impression[pid])
        else:
            num = 0
            a = itertools.combinations(imgs, 2)
            for i in a:
                image1.append(i[0])
                image2.append(i[1])
                person_id.append(pid + '_' + str(num))
                report.append(findings[pid])
                result.append(impression[pid])
                num += 1
    data = pd.DataFrame()
    data['Person_id'] = person_id
    data['Image1'] = image1
    data['Image2'] = image2
    data['Report'] = report
    data['Result'] = result
    return data

In [ ]:
train = create_data(img_per_person_train)
test = create_data(img_per_person_test)
cv = create_data(img_per_person_cv)